In [371]:
from typing import List
import os
import re
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
import requests

In [372]:
import os

from util.widgets import generate_ym, generate_ymd, generate_ymd_HMS, generate_i

ym = generate_ym()
g = 'cargohose_herren'
bd = f'{g}/{ym}'
bd_a = f'{bd}/a'
b_d = f'{g}_{ym}'

os.makedirs(bd, exist_ok=True)
os.makedirs(bd_a, exist_ok=True)

ymd = generate_ymd()
ymd_HMS = generate_ymd_HMS()
i = '(08-09)'
bd_a_img = f'{bd_a}/{ymd}'
os.makedirs(bd_a_img, exist_ok=True)

In [373]:
fp_erp = f'{bd_a}/product_20240729110524847.xls'
erp_df: DataFrame = pd.read_excel(fp_erp)

erp_df

,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,S3图片链接 8缩略图,S3图片链接 8,S3图片链接 9缩略图,S3图片链接 9,子图链接,长,宽,高,尺码表,印花名称
0,AHY240625001,NaN,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Yellow,Green,Coffee,Khaki,Grey","S,M,L,XL,XXL,XXXL",NaN,...,NaN,https://oz1xbcxa3ey1u5sz.imgsapp.com/afsd/1935...,NaN,https://o09tclpn5hvfql24.imgsapp.com/afsd/1935...,NaN,30,20,3,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,其他
1,AHY240625001BKL,AHY240625001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤 黑色 L-31,"<h1 align=""center""><span style=""font-size:14pt...",Black,L-31,NaN,...,NaN,NaN,NaN,NaN,NaN,33,24,1,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,NaN
2,AHY240625001BKL2,AHY240625001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤 黑色XXL-33,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXL-33,NaN,...,NaN,NaN,NaN,NaN,NaN,33,24,1,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,NaN
3,AHY240625001BKL3,AHY240625001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤 黑色 XXXL-34,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXXL-34,NaN,...,NaN,NaN,NaN,NaN,NaN,33,24,1,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,NaN
4,AHY240625001BKM,AHY240625001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Male Summer Cargo Pants Solid Color Pocket Tet...,男士多口袋纽扣五分工装短裤 黑色 M-30,"<h1 align=""center""><span style=""font-size:14pt...",Black,M-30,NaN,...,NaN,NaN,NaN,NaN,NaN,33,24,1,Size: S Waist: 74cm/29.13'' Hip: 121cm/47.64''...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,ZZS210511744GYL5,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 XXXXXL,"<h1 align=""center""><span style=""font-size:14pt...",Grey,XXXXXL,5X-Large,...,NaN,https://beaqdilces304it4.imgsapp.com/afsd/8802...,NaN,https://xmvlodrvwlswrqur.imgsapp.com/afsd/8802...,NaN,30,23,1,Size: S US: 4 UK: 8 EU: 34 Waist: 77cm/30.31''...,NaN
684,ZZS210511744GYS,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 S,"<h1 align=""center""><span style=""font-size:14pt...",Grey,S,Small,...,NaN,https://uw04vmkwhu29sfli.imgsapp.com/afsd/8802...,NaN,https://yehpc88ffcmmurw8.imgsapp.com/afsd/8802...,NaN,30,23,1,Size: S US: 4 UK: 8 EU: 34 Waist: 77cm/30.31''...,NaN
685,ZZS210511744KHL4,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 卡其色 XXXXL,"<h1 align=""center""><span style=""font-size:14pt...",Khaki,XXXXL,4X-Large,...,NaN,https://oz7ag1f8cmj8r9t2.imgsapp.com/afsd/8802...,NaN,https://4g2e6dzvc73tya1m.imgsapp.com/afsd/8802...,NaN,30,23,1,Size: S US: 4 UK: 8 EU: 34 Waist: 77cm/30.31''...,NaN
686,ZZS210511744KHL5,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 卡其色 XXXXXL,"<h1 align=""center""><span style=""font-size:14pt...",Khaki,XXXXXL,5X-Large,...,NaN,https://pxqulwuv6hvf1d9p.imgsapp.com/afsd/8802...,NaN,https://idkvyw32unyuuhqc.imgsapp.com/afsd/8802...,NaN,30,23,1,Size: S US: 4 UK: 8 EU: 34 Waist: 77cm/30.31''...,NaN


# 1. 生成amazon_sku, amazon_psku, parent_child, 过滤无用数据行

In [374]:
def generate_amazon_sku(s: str, c: str, n: str, b: int) -> str:
    """

    :param s: ERP的sku
    :param c: 仓库编号
    :param n: 节点
    :param b: 父sku批号
    :return:
    """
    p = re.compile(r'^(?P<p>[A-Z]+)(?P<d>\d{6})(?P<n>\d+)(?P<cs>.*?)$', re.S)
    # for i in p.match(s).groups():
    #     print(i)
    t = ymd_HMS[2:].replace(' ', '-').replace(':', '-')
    cs = p.match(s).group('cs')
    result = f'{c}-{n}-{t}-{b:03d}-{cs}'.strip('-')
    return result


s1 = 'BYJ240716009BWL2'
# MKT-DE-24-07-18-001-BWL2
generate_amazon_sku(s1, 'FRL', 'DE', 1)

'FRL-DE-24-07-29-21-02-41-001-BWL2'

In [375]:
erp_df['父SKU'] = erp_df['父SKU'].fillna(erp_df['SKU']).copy()
erp_df['parent_child'] = np.where(erp_df['父SKU'] == erp_df['SKU'], 'Parent', 'Child')

In [376]:
ams = []
amps = []

for i, (n, group) in enumerate(erp_df.groupby(by=['父SKU'])):
    for index, row in group.iterrows():
        am_sku = generate_amazon_sku(row['SKU'], 'FRL', 'DE', i + 1)
        am_psku = generate_amazon_sku(row['父SKU'], 'FRL', 'DE', i + 1)
        ams.append(am_sku)
        amps.append(am_psku)

erp_df['amazon_sku'] = ams.copy()
erp_df['amazon_psku'] = amps.copy()
erp_df[['SKU', '父SKU', 'amazon_sku', 'amazon_psku', 'parent_child']]

,SKU,父SKU,amazon_sku,amazon_psku,parent_child
0,AHY240625001,AHY240625001,FRL-DE-24-07-29-21-02-41-001,FRL-DE-24-07-29-21-02-41-001,Parent
1,AHY240625001BKL,AHY240625001,FRL-DE-24-07-29-21-02-41-001-BKL,FRL-DE-24-07-29-21-02-41-001,Child
2,AHY240625001BKL2,AHY240625001,FRL-DE-24-07-29-21-02-41-001-BKL2,FRL-DE-24-07-29-21-02-41-001,Child
3,AHY240625001BKL3,AHY240625001,FRL-DE-24-07-29-21-02-41-001-BKL3,FRL-DE-24-07-29-21-02-41-001,Child
4,AHY240625001BKM,AHY240625001,FRL-DE-24-07-29-21-02-41-001-BKM,FRL-DE-24-07-29-21-02-41-001,Child
...,...,...,...,...,...
683,ZZS210511744GYL5,ZZS210511744,FRL-DE-24-07-29-21-02-41-050-GYL5,FRL-DE-24-07-29-21-02-41-050,Child
684,ZZS210511744GYS,ZZS210511744,FRL-DE-24-07-29-21-02-41-050-GYS,FRL-DE-24-07-29-21-02-41-050,Child
685,ZZS210511744KHL4,ZZS210511744,FRL-DE-24-07-29-21-02-41-050-KHL4,FRL-DE-24-07-29-21-02-41-050,Child
686,ZZS210511744KHL5,ZZS210511744,FRL-DE-24-07-29-21-02-41-050-KHL5,FRL-DE-24-07-29-21-02-41-050,Child


In [377]:
inuse_skus_path = './util/skus/inuse_skus.txt'
archived_skus_path = './util/skus/archived_skus.txt'
deleted_skus_path = './util/skus/deleted_skus.txt'
recalled_skus_path = './util/skus/recalled_skus.txt'
other_skus_path = './util/skus/other_skus.txt'
# inuse_skus = open(inuse_skus_path, mode='r', encoding='utf-8').readlines()
# inuse_skus = [i.strip() for i in inuse_skus]
# archived_skus = open(archived_skus_path, mode='r', encoding='utf-8').readlines()
# archived_skus = [i.strip() for i in archived_skus]
# deleted_skus = open(deleted_skus_path, mode='r', encoding='utf-8').readlines()
# deleted_skus = [i.strip() for i in deleted_skus]
# recalled_skus = open(recalled_skus_path, mode='r', encoding='utf-8').readlines()
# recalled_skus = [i.strip() for i in recalled_skus]
# other_skus = open(other_skus_path, mode='r', encoding='utf-8').readlines()
# other_skus = [i.strip() for i in other_skus]
#
# erp_df = erp_df[~erp_df['父SKU'].isin(inuse_skus + archived_skus + deleted_skus + recalled_skus + other_skus)]

# 过滤掉无用数据行

"需要手动操作，过滤下架的品(Deleted)，导出的文件还要检查
1. 淘汰掉那些与目标品类（长裤，短裤）不一样的品(Other)
2. 父sku是否有子sku(Archived)
3. 有无图片缺失(Archived)"


In [378]:
erp_df['父SKU'].unique()

array(['AHY240625001', 'ARH230522004', 'AWB231219002', 'CMG210615252',
       'CMG210906252', 'CMG220307256', 'CMG220905004', 'CMG221207014',
       'CMG221207015', 'CMG221207016', 'CMG221207017', 'CMG221207020',
       'CMG221207021', 'CMG230302006', 'CWL231026005', 'FBE220924043',
       'FZQ230705020', 'FZQ230925006', 'FZQ230925008', 'HCJ231123001',
       'LBB230904023', 'LBB230904027', 'LBB230904028', 'LBB230904029',
       'LBB230904031', 'LBB240123002', 'LJH81008337', 'LLL231122001',
       'LLL71218030', 'LNP220809008', 'LZZ220310635', 'MJJ201023291',
       'QXP240705001', 'SCN221128001', 'SFW211229313', 'SQI220926005',
       'WMQ230306012', 'WMQ230616009', 'YAY81204372', 'YLA231009004',
       'YWZ240104009', 'ZME220405267', 'ZME220827007', 'ZTY210623248',
       'ZTY230213002', 'ZTY230401011', 'ZTY230829002', 'ZTY230829005',
       'ZTY231030001', 'ZZS210511744'], dtype=object)

## Archived

In [379]:
archived_skus_path = './util/skus/archived_skus.txt'
# 读取已存档的SKUs
with open(archived_skus_path, 'r') as file:
    archived_skus = set(line.strip() for line in file)

# 第一步: 记录并删除`父SKU`字段值唯一的行
unique_psku_indexes = erp_df['父SKU'].drop_duplicates(keep=False).index
unique_pskus = erp_df.loc[unique_psku_indexes, '父SKU']
erp_df.drop(unique_psku_indexes, inplace=True)

# 第二步: 记录并删除`图片链接`字段值为 NaN 的行
nan_image_links_indexes = erp_df['图片链接 1'].isna()
nan_image_links_pskus = erp_df.loc[nan_image_links_indexes, '父SKU']
# erp_df.drop(nan_image_links_indexes.index, inplace=True)

# 合并两个集合
to_archive_skus = unique_pskus.append(nan_image_links_pskus)

# 将新`父SKU`添加到文件末尾（如果不存在于已存档的SKU中）
with open(archived_skus_path, 'a') as file:
    for sku in to_archive_skus.unique():
        if sku not in archived_skus:
            file.write(f"{sku}\n")

to_archive_skus.unique()

array(['ARH230522004', 'LBB230904028', 'CMG210615252', 'LJH81008337',
       'LLL71218030', 'YAY81204372'], dtype=object)

## Other

In [380]:
import pandas as pd

# 假设 erp_df 已经是一个加载好的DataFrame
# 如果没有，请先加载你的数据
# erp_df = pd.read_csv('your_data.csv')

other_skus_path = './util/skus/other_skus.txt'

# 读取已存档的SKUs
with open(other_skus_path, 'r') as file:
    other_skus = set(line.strip() for line in file)

minors = ['短', '短裤', '7分']

# 第一步: 记录并删除`产品名称`字段值包含“短”或“短裤”的行
other_rows = erp_df['产品名称'].str.contains('|'.join(minors), case=False, regex=True)
other_pskus = erp_df.loc[other_rows, '父SKU']
erp_df.drop(erp_df.index[other_rows], inplace=True)

# 第二步: 将新`父SKU`添加到文件末尾（如果不存在于已存档的SKU中）
with open(other_skus_path, 'a') as file:
    for sku in other_pskus.unique():
        if sku not in other_skus:
            file.write(f"{sku}\n")

other_pskus.unique()

array(['AHY240625001', 'CMG210615252', 'LBB240123002', 'LLL71218030',
       'LZZ220310635', 'WMQ230306012', 'YWZ240104009', 'ZME220405267'],
      dtype=object)

In [381]:
erp_df['父SKU'].unique()

array(['AWB231219002', 'CMG210906252', 'CMG220307256', 'CMG220905004',
       'CMG221207014', 'CMG221207015', 'CMG221207016', 'CMG221207017',
       'CMG221207020', 'CMG221207021', 'CMG230302006', 'CWL231026005',
       'FBE220924043', 'FZQ230705020', 'FZQ230925006', 'FZQ230925008',
       'HCJ231123001', 'LBB230904023', 'LBB230904027', 'LBB230904029',
       'LBB230904031', 'LJH81008337', 'LLL231122001', 'LNP220809008',
       'MJJ201023291', 'QXP240705001', 'SCN221128001', 'SFW211229313',
       'SQI220926005', 'WMQ230616009', 'YAY81204372', 'YLA231009004',
       'ZME220827007', 'ZTY210623248', 'ZTY230213002', 'ZTY230401011',
       'ZTY230829002', 'ZTY230829005', 'ZTY231030001', 'ZZS210511744'],
      dtype=object)

# 2. 生成title

In [382]:
fp_title = f'{g}/{g}_title.xlsx'
title_df: DataFrame = pd.read_excel(fp_title, sheet_name='title')

from util.widgets import Status

title_unused_df = title_df[title_df['status'] == Status.UNUSED.value]

# 创建一个字典来跟踪每个 parent_sku 分配的 title
title_allocation = {}

# 遍历 title_df，按顺序为每个 parent_sku 分配 title
for i, row in title_unused_df.iterrows():
    title = row['title']
    # 如果所有的 parent_sku 都已分配 title，则跳出循环
    if set(title_allocation.keys()) == set(erp_df['父SKU'].unique()):
        break

    # 找到第一个尚未分配 title 的 parent_sku
    for parent_sku in erp_df['父SKU'].unique():
        if parent_sku not in title_allocation:
            title_allocation[parent_sku] = title
            break

# 更新 title_df 中被分配的 title 的状态
title_unused_df.loc[title_unused_df['title'].isin(title_allocation.values()), 'status'] = Status.IN_USE.value

# 将分配的 title 添加到 erp_df 中
erp_df['title'] = erp_df['父SKU'].map(title_allocation)
index = title_unused_df[title_unused_df['status'] == Status.IN_USE.value].index
title_df.iloc[index, title_df.columns.get_loc('status')] = Status.IN_USE.value
title_df.iloc[index, title_df.columns.get_loc('ymd_HMS')] = ymd_HMS
title_df

e:\aenv\python-3.8.0(64-bit)\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\000\AppData\Local\Temp\ipykernel_7376\340360046.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_unused_df.loc[title_unused_df['title'].isin(title_allocation.values()), 'status'] = Status.IN_USE.value


,title,ymd,status,ymd_HMS
0,Cargohose Herren Cargohosen Hose Men Sommer He...,2024_07_26,InUse,2024-07-26 15:53:25
1,Cargohose Herren Cargo Hose Men Sommer Lange H...,2024_07_26,InUse,2024-07-26 15:53:25
2,Cargohose Herren Cargo Hosen Men Sommer Cargoh...,2024_07_26,InUse,2024-07-26 15:53:25
3,Cargohose Herren Lang Sommer Hose Herren Leich...,2024_07_26,InUse,2024-07-26 15:53:25
4,Cargohose Herren Lang Sommer Herren Hose Baggy...,2024_07_26,InUse,2024-07-26 15:53:25
...,...,...,...,...
95,Cargohose Herren Schwarze Cargo Men Sommerhose...,2024_07_26,Unused,NaN
96,Cargohose Herren Cargo Hose Pants Men Herren H...,2024_07_26,Unused,NaN
97,Cargohose Herren Pants Men Sommerhosen Schwarz...,2024_07_26,Unused,NaN
98,Cargohose Herren Lang Sommer Leichte Arbeitsho...,2024_07_26,Unused,NaN


# 3. 生成st250

In [383]:
fp_st250 = f'{g}/{g}_st250.xlsx'
st250_df: DataFrame = pd.read_excel(fp_st250, sheet_name='st250')

from util.widgets import Status

st250_unused_df = st250_df[st250_df['status'] == Status.UNUSED.value]

# 创建一个字典来跟踪每个 parent_sku 分配的 st250
st250_allocation = {}

# 遍历 st250_df，按顺序为每个 parent_sku 分配 st250
for i, row in st250_unused_df.iterrows():
    value = row['value']
    # 如果所有的 parent_sku 都已分配 title，则跳出循环
    if set(st250_allocation.keys()) == set(erp_df['父SKU'].unique()):
        break

    # 找到第一个尚未分配 title 的 parent_sku
    for parent_sku in erp_df['父SKU'].unique():
        if parent_sku not in st250_allocation:
            st250_allocation[parent_sku] = value
            break

# 更新 st250_df 中被分配的 title 的状态
st250_unused_df.loc[st250_unused_df['value'].isin(st250_allocation.values()), 'status'] = Status.IN_USE.value

# 将分配的 title 添加到 erp_df 中
erp_df['st250'] = erp_df['父SKU'].map(st250_allocation)
index = st250_unused_df[st250_unused_df['status'] == Status.IN_USE.value].index
st250_df.iloc[index, st250_df.columns.get_loc('status')] = Status.IN_USE.value
st250_df.iloc[index, st250_df.columns.get_loc('ymd_HMS')] = ymd_HMS
st250_df

e:\aenv\python-3.8.0(64-bit)\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,group,parent,len,value,ymd,status,ymd_HMS
0,cargohose herren,cargohose herren,240,cargohose herren baggy cargohose herren sommer...,2024_07_26,InUse,2024-07-26 15:53:25
1,cargohose herren,cargohose herren,231,cargohose herren stretch cargohose herren somm...,2024_07_26,InUse,2024-07-26 15:53:25
2,cargohose herren,cargohose herren,244,cargohose herren schwarz cargohose herren somm...,2024_07_26,InUse,2024-07-26 15:53:25
3,cargohose herren,cargohose herren,237,cargohose herren stretch cargohose herren schw...,2024_07_26,InUse,2024-07-26 15:53:25
4,cargohose herren,cargohose herren,248,cargohose herren stretch cargohose herren stre...,2024_07_26,InUse,2024-07-26 15:53:25
...,...,...,...,...,...,...,...
104,cargohose herren,cargohose herren stretch,246,cargohose herren stretch slim fit cargohose he...,2024_07_26,Unused,NaN
105,cargohose herren,cargohose herren stretch,247,cargohose herren stretch sommer cargohose herr...,2024_07_26,Unused,NaN
106,cargohose herren,cargohose herren stretch,245,cargohose herren stretch übergröße cargohose h...,2024_07_26,Unused,NaN
107,cargohose herren,cargohose herren stretch,242,cargohose herren stretch lang cargohose herren...,2024_07_26,Unused,NaN


# 4. 生成product_description

In [384]:
"""
面料/材质：
特征/设计：
搭配：
适用场合：
客户服务：

【轻质材料】优质涤纶。轻盈、柔软亲肤，舒适、透气、轻便，即使在夏季也能舒适慢跑。
【实用设计】运动修身健身运动裤，具有良好的弹性、透气性、重量轻、无摩擦和刺激，腿部贴合，展现肌肉张力并提供最大的运动自由度。
【易于搭配】我们的男士工装裤可以与夹克、毛衣等搭配，冬天保暖又时尚。这款裤子也可以搭配T恤和衬衫。一年四季必备的裤子。
【更多用途】适合休闲、运动、旅行、户外、家居、其他非正式活动或商务场合等，也可以在日常生活和工作中穿着。这款裤子也是送给朋友、儿子、丈夫或自己的完美礼物。
【注意】订购前，请参考产品说明中我们的详细尺码表（不是亚马逊尺码表）。由于手工测量，请允许1-2厘米的误差。我们尽力向您展示正确的颜色，但请接受由于显示器和灯光不同而出现的一些轻微色差。

描述：需要转成html格式
【特征】
材质：聚酯纤维
包装内容：1 条工装裤
风格: 休闲
性别：男
合身：宽松
长度：长裤
季节: 春季，夏季，秋季，冬季
【尺寸信息】
erp_df['尺码表']
【客户服务】
我们致力于为客户提供舒适的购物体验。如果您有任何疑问，请写信给我们，您将得到完美的解决方案。
"""

bullet_point1 = '【Leichtes Material】Hochwertiges Polyester. Leicht, weich und hautfreundlich, komfortabel, atmungsaktiv und leicht, ideal auch für das Joggen im Sommer.'
bullet_point2 = '【Praktisches Design】Sportliche, figurbetonte Trainingshose mit guter Elastizität, Atmungsaktivität, leichtem Gewicht, ohne Reibung und Irritationen. Sie passt sich eng an die Beine an, zeigt die Muskelspannung und bietet maximale Bewegungsfreiheit.'
bullet_point3 = '【Einfach zu kombinieren】Unsere Herren Cargohose kann mit Jacken, Pullovern usw. kombiniert werden, um im Winter warm und modisch zu bleiben. Diese Hose kann auch mit T-Shirts und Hemden getragen werden. Eine Hose, die das ganze Jahr über ein Muss ist.'
bullet_point4 = '【Vielfältige Verwendungsmöglichkeiten】Geeignet für Freizeit, Sport, Reisen, Outdoor, Zuhause, andere informelle Anlässe oder Geschäftstreffen. Diese Hose ist auch ein perfektes Geschenk für Freunde, Söhne, Ehemänner oder sich selbst.'
bullet_point5 = '【Hinweis】Bitte beachten Sie vor der Bestellung unsere detaillierte Größentabelle in der Produktbeschreibung (nicht die Amazon-Größentabelle). Aufgrund manueller Messungen erlauben Sie bitte einen Fehler von 1-2 cm. Wir bemühen uns, Ihnen die richtigen Farben zu zeigen, aber akzeptieren Sie bitte leichte Farbabweichungen aufgrund unterschiedlicher Monitore und Beleuchtung.'


def generate_pd(t: str):
    """
    Größe: S Taille: 78-84cm/30.71-33.07'' Hüfte: 100cm/39.37'' Länge: 66cm/25.98''
    :param t:
    :return:
    """
    try:
        de_map = {
            'Size': 'Größe',
            'Waist': 'Taille',
            'Hip': 'Hüfte',
            'Length': 'Länge',
        }
        for k, v in de_map.items():
            t = t.replace(k, v)
        cs = t.split('\n')
    except AttributeError:
        cs = []
    pd = f"""<p><b>Merkmale</b><br></p>
<p>Material: Polyester</p>
<p>Verpackungsinhalt: 1 Cargo-Hose</p>
<p>Stil: Freizeit</p>
<p>Geschlecht: Herren</p>
<p>Passform: Locker</p>
<p>Länge: Lange Hose</p>
<p>Saison: Frühling, Sommer, Herbst, Winter</p>
<p><b>Größeninformationen</b></p>
{''.join([f'<p>{c.strip()}</p>' for c in cs])}
<p><b>Kundendienst</b></p>
<p>Wir sind bestrebt, unseren Kunden ein komfortables Einkaufserlebnis zu bieten. Wenn Sie Fragen haben, schreiben Sie uns bitte, und Sie erhalten eine perfekte Lösung.</p>"""
    return pd.replace('\n', '')


erp_df['bullet_point1'] = bullet_point1
erp_df['bullet_point2'] = bullet_point2
erp_df['bullet_point3'] = bullet_point3
erp_df['bullet_point4'] = bullet_point4
erp_df['bullet_point5'] = bullet_point5
erp_df['product_description'] = erp_df['尺码表'].apply(lambda i: generate_pd(i))
erp_df

,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,amazon_sku,amazon_psku,title,st250,bullet_point1,bullet_point2,bullet_point3,bullet_point4,bullet_point5,product_description
38,AWB231219002,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤,"<h1 align=""center""><span style=""font-size:14pt...","Green,Red","M,L,XL,XXL,XXXL",NaN,...,FRL-DE-24-07-29-21-02-41-003,FRL-DE-24-07-29-21-02-41-003,Cargohose Herren Outdoor Hose Hosen Herren Out...,cargohose herren baggy cargohose herren sommer...,【Leichtes Material】Hochwertiges Polyester. Lei...,"【Praktisches Design】Sportliche, figurbetonte T...",【Einfach zu kombinieren】Unsere Herren Cargohos...,【Vielfältige Verwendungsmöglichkeiten】Geeignet...,【Hinweis】Bitte beachten Sie vor der Bestellung...,<p><b>Merkmale</b><br></p><p>Material: Polyest...
39,AWB231219002GNL,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 L,"<h1 align=""center""><span style=""font-size:14pt...",Green,L,Large,...,FRL-DE-24-07-29-21-02-41-003-GNL,FRL-DE-24-07-29-21-02-41-003,Cargohose Herren Outdoor Hose Hosen Herren Out...,cargohose herren baggy cargohose herren sommer...,【Leichtes Material】Hochwertiges Polyester. Lei...,"【Praktisches Design】Sportliche, figurbetonte T...",【Einfach zu kombinieren】Unsere Herren Cargohos...,【Vielfältige Verwendungsmöglichkeiten】Geeignet...,【Hinweis】Bitte beachten Sie vor der Bestellung...,<p><b>Merkmale</b><br></p><p>Material: Polyest...
40,AWB231219002GNL2,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 XXL,"<h1 align=""center""><span style=""font-size:14pt...",Green,XXL,XX-Large,...,FRL-DE-24-07-29-21-02-41-003-GNL2,FRL-DE-24-07-29-21-02-41-003,Cargohose Herren Outdoor Hose Hosen Herren Out...,cargohose herren baggy cargohose herren sommer...,【Leichtes Material】Hochwertiges Polyester. Lei...,"【Praktisches Design】Sportliche, figurbetonte T...",【Einfach zu kombinieren】Unsere Herren Cargohos...,【Vielfältige Verwendungsmöglichkeiten】Geeignet...,【Hinweis】Bitte beachten Sie vor der Bestellung...,<p><b>Merkmale</b><br></p><p>Material: Polyest...
41,AWB231219002GNL3,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 XXXL,"<h1 align=""center""><span style=""font-size:14pt...",Green,XXXL,3X-Large,...,FRL-DE-24-07-29-21-02-41-003-GNL3,FRL-DE-24-07-29-21-02-41-003,Cargohose Herren Outdoor Hose Hosen Herren Out...,cargohose herren baggy cargohose herren sommer...,【Leichtes Material】Hochwertiges Polyester. Lei...,"【Praktisches Design】Sportliche, figurbetonte T...",【Einfach zu kombinieren】Unsere Herren Cargohos...,【Vielfältige Verwendungsmöglichkeiten】Geeignet...,【Hinweis】Bitte beachten Sie vor der Bestellung...,<p><b>Merkmale</b><br></p><p>Material: Polyest...
42,AWB231219002GNM,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 M,"<h1 align=""center""><span style=""font-size:14pt...",Green,M,Medium,...,FRL-DE-24-07-29-21-02-41-003-GNM,FRL-DE-24-07-29-21-02-41-003,Cargohose Herren Outdoor Hose Hosen Herren Out...,cargohose herren baggy cargohose herren sommer...,【Leichtes Material】Hochwertiges Polyester. Lei...,"【Praktisches Design】Sportliche, figurbetonte T...",【Einfach zu kombinieren】Unsere Herren Cargohos...,【Vielfältige Verwendungsmöglichkeiten】Geeignet...,【Hinweis】Bitte beachten Sie vor der Bestellung...,<p><b>Merkmale</b><br></p><p>Material: Polyest...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,ZZS210511744GYL5,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 XXXXXL,"<h1 align=""c

# 5. 生成size, color

In [385]:
# 选择parent_child字段值为'Parent'的行
is_parent = erp_df['parent_child'] == 'Parent'
c_s = re.compile(r'(?P<s>\d{2}|\d[A-Z]|[A-Z]{1,2})(?P<c>\d[A-Z]{2}|[A-Z]{2})')

color_map = {
    'AG': 'Armeegrün',  # AG
    'BG': 'Beige',  # Beige
    'BK': 'Schwarz',  # Black
    'BU': 'Blau',  # Blue
    'BW': 'Braun',  # Brown
    'CE': 'Camo',  # Camouflage
    'CO': 'Kaffee',  # Coffee
    'DB': 'Dunkelblau',  # Dark Blue
    'DY': 'Dunkelgrau',  # Dark Gray
    'GN': 'Grün',  # Green
    'GY': 'Grau',  # Grey
    'GY1': 'Grau-1',  # Grey
    'GY2': 'Grau-2',  # Grey
    'LN': 'Hellgrün',  # Light Green
    'LR': 'Hellgrau',  # Light Gray
    'KH': 'Khaki',  # Khaki
    'NY': 'Navy',  # Navy
    'RD': 'Rot',  # Red
    'WE': 'Wine',  # Wine
    'WH': 'WeiB',  # White
    'YE': 'Gelb',  # Yellow
}


def get_color(r: str):
    try:
        c = c_s.match(r[::-1]).group('c')[::-1]
    except AttributeError:
        c = ''
    try:
        clr = color_map[c]
    except KeyError:
        clr = c

    return clr


size_map = {
    'S': 'S',
    'M': 'M',
    'L': 'L',
    'XL': 'XL',
    'L1': 'XL',
    'L2': 'XXL',
    'L3': '3XL',
    'L4': '4XL',
    'L5': '5XL',
    'L6': '6XL',
}


def get_size(r: str):
    try:  # L, L2, 23, ''
        s = c_s.match(r[::-1]).group('s')[::-1]
    except AttributeError:
        s = ''
    try:
        se = size_map[s]
    except KeyError:
        if s:
            se = int(s)
        else:
            se = s

    return se


s2 = 'MKT-DE-24-07-18-001-BKL1'
get_color(s2), get_size(s2)

('Schwarz', 'XL')

In [386]:
erp_df['size'] = erp_df['amazon_sku'].apply(lambda i: get_size(i))
erp_df['color'] = erp_df['amazon_sku'].apply(lambda i: get_color(i))

# 6. 生成图片链接

In [387]:
# img_home = f'https://www.gitee.com/wuku32/project-step-2/{bd_a_img}/'
# 'https://raw.githubusercontent.com/wuku32/farewellEvony/master/img/img_11.png'
# img_home = f'https://www.gitee.com/wuku32/project-step-2/raw/master/{bd_a_img}/'
img_home = f'https://raw.githubusercontent.com/wuku32/project-step-2/master/{bd_a_img}/'


def redirect(u: str):
    try:
        u = re.sub(r'http://.*/', img_home, u)
        return u
    except TypeError:
        pass


s1 = [f'图片链接 {i}' for i in range(1, 10)]
for _ in s1:
    erp_df[f'github{_}'] = erp_df[_].apply(lambda i: redirect(i))

erp_df

,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,color,github图片链接 1,github图片链接 2,github图片链接 3,github图片链接 4,github图片链接 5,github图片链接 6,github图片链接 7,github图片链接 8,github图片链接 9
38,AWB231219002,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤,"<h1 align=""center""><span style=""font-size:14pt...","Green,Red","M,L,XL,XXL,XXXL",NaN,...,,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None
39,AWB231219002GNL,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 L,"<h1 align=""center""><span style=""font-size:14pt...",Green,L,Large,...,Grün,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,None,None,None
40,AWB231219002GNL2,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 XXL,"<h1 align=""center""><span style=""font-size:14pt...",Green,XXL,XX-Large,...,Grün,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,None,None,None
41,AWB231219002GNL3,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 XXXL,"<h1 align=""center""><span style=""font-size:14pt...",Green,XXXL,3X-Large,...,Grün,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,None,None,None
42,AWB231219002GNM,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 M,"<h1 align=""center""><span style=""font-size:14pt...",Green,M,Medium,...,Grün,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,ZZS210511744GYL5,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 XXXXXL,"<h1 align=""center""><span style=""font-size:14pt...",Grey,XXXXXL,5X-Large,...,Grau,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...
684,ZZS210511744GYS,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 S,"<h1 align=""center""><span style=""font-size:14pt...",Grey,S,Small,...,Grau,https://raw.githubusercontent.com/wuku32/proje...,https:

# 7. 生成standard_price

In [388]:
def get_p(text: str):
    a = re.compile(r'.*?欧洲_挂号价.*?：(?P<rp>.*?) EUR')
    rp = a.match(text).group('rp')
    x = 1.6
    return float(int(float(rp) / x + 0.5) - 0.01)


erp_df['standard_price'] = erp_df['亚马逊建议价'].apply(lambda i: get_p(i))
erp_df

,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,github图片链接 1,github图片链接 2,github图片链接 3,github图片链接 4,github图片链接 5,github图片链接 6,github图片链接 7,github图片链接 8,github图片链接 9,standard_price
38,AWB231219002,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤,"<h1 align=""center""><span style=""font-size:14pt...","Green,Red","M,L,XL,XXL,XXXL",NaN,...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,14.99
39,AWB231219002GNL,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 L,"<h1 align=""center""><span style=""font-size:14pt...",Green,L,Large,...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,None,None,None,13.99
40,AWB231219002GNL2,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 XXL,"<h1 align=""center""><span style=""font-size:14pt...",Green,XXL,XX-Large,...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,None,None,None,14.99
41,AWB231219002GNL3,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 XXXL,"<h1 align=""center""><span style=""font-size:14pt...",Green,XXXL,3X-Large,...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,None,None,None,14.99
42,AWB231219002GNM,AWB231219002,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Fitness Sweatpants Stretchy Sport Elastic Body...,新款夏季休闲运动系绳七分裤男士时尚修身健身运动裤束脚裤 绿色 M,"<h1 align=""center""><span style=""font-size:14pt...",Green,M,Medium,...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,None,None,None,None,14.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,ZZS210511744GYL5,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 XXXXXL,"<h1 align=""center""><span style=""font-size:14pt...",Grey,XXXXXL,5X-Large,...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,https://raw.githubusercontent.com/wuku32/proje...,31.99
684,ZZS210511744GYS,ZZS210511744,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Casual Straight Summer Multi Pocket Men's Pant...,男士夏季多口袋直筒休闲裤工装裤长裤 灰色 S,"<h1 align=""center""><span style=""font-size:14pt...",Grey,S,Small,...,https://raw.githubusercontent.com/wuku32/p

In [389]:
erp_df['父SKU'].unique()

array(['AWB231219002', 'CMG210906252', 'CMG220307256', 'CMG220905004',
       'CMG221207014', 'CMG221207015', 'CMG221207016', 'CMG221207017',
       'CMG221207020', 'CMG221207021', 'CMG230302006', 'CWL231026005',
       'FBE220924043', 'FZQ230705020', 'FZQ230925006', 'FZQ230925008',
       'HCJ231123001', 'LBB230904023', 'LBB230904027', 'LBB230904029',
       'LBB230904031', 'LJH81008337', 'LLL231122001', 'LNP220809008',
       'MJJ201023291', 'QXP240705001', 'SCN221128001', 'SFW211229313',
       'SQI220926005', 'WMQ230616009', 'YAY81204372', 'YLA231009004',
       'ZME220827007', 'ZTY210623248', 'ZTY230213002', 'ZTY230401011',
       'ZTY230829002', 'ZTY230829005', 'ZTY231030001', 'ZZS210511744'],
      dtype=object)

# 手动操作！！！

In [391]:
# erp_df.to_excel(fp_erp.replace('.xls', '_verified.xls'), index=False)
#
# with open(inuse_skus_path, 'r') as file:
#     inuse_skus = set(line.strip() for line in file)
#
# with open(inuse_skus_path, 'a') as file:
#     file.write(f"{ymd_HMS}\n")
#     for sku in erp_df['父SKU'].unique():
#         if sku not in inuse_skus:
#             file.write(f"{sku}\n")
#
# other_pskus.unique()
#
# fp_tgt = f'{g}/{g}_tgt.xlsx'
# tgt_df: DataFrame = pd.read_excel(fp_tgt, sheet_name='tgt')
# # 创建一个映射字典，将父SKU映射到对应的amazon_psku
# mapping_dict = erp_df.set_index('父SKU')['amazon_psku'].to_dict()
# # 使用map函数将父SKU映射到amazon_psku
# tgt_df['amazon_psku'] = tgt_df['productSku'].map(mapping_dict)
# tgt_df.loc[tgt_df['productSku'].isin(erp_df['父SKU'].unique()), 'status'] = Status.IN_USE.value
# tgt_df.loc[tgt_df['productSku'].isin(erp_df['父SKU'].unique()), 'ymd_HMS'] = ymd_HMS
# tgt_df.to_excel(fp_tgt, index=False)
#
# # 将修改后的DataFrame写回到Excel文件中的特定工作簿
#
# from openpyxl import load_workbook
#
# # 加载现有的工作簿
# book = load_workbook(fp_title)
#
# # 使用openpyxl的方式选择要修改的工作表
# with pd.ExcelWriter(fp_title, engine='openpyxl') as writer:
#     writer.book = book
#
#     if 'title' in book.sheetnames:
#         book.remove(book['title'])
#
#     # 将修改后的工作表写回工作簿
#     title_df.to_excel(writer, index=False, sheet_name='title')
#
# book = load_workbook(fp_st250)
#
# # 使用openpyxl的方式选择要修改的工作表
# with pd.ExcelWriter(fp_st250, engine='openpyxl') as writer:
#     writer.book = book
#
#     if 'st250' in book.sheetnames:
#         book.remove(book['st250'])
#
#     # 将修改后的工作表写回工作簿
#     st250_df.to_excel(writer, index=False, sheet_name='st250')